In [1]:
import re
from tqdm import tqdm
import numpy as npd
import pandas as pd
from math import ceil

In [2]:
DF = pd.read_csv('data/data_lower.csv', sep='\t', encoding = 'utf-8')

In [3]:
#создадим список классов по частотности
df_classes = DF[['classtag']].copy()
df_classes['total_count'] = [1] * df_classes.shape[0]
df_classes = df_classes.groupby(['classtag'], sort=True).aggregate(sum) #создаем таблицу класс * количество парадигм
df_classes = df_classes.sort_values(by=['total_count'], ascending=False) #сортируем от большего к меньшему
df_classes = df_classes.reset_index(level=['classtag'])
cl_list = list(df_classes['classtag']) # сам список классов по убыванию
print('всего классов',df_classes.shape[0])
print(df_classes[:5])


всего классов 433
  classtag  total_count
0      м1а         7647
1      ж8а         3747
2      с7а         3625
3     ж3*а         3606
4     мо1а         2891


In [4]:
tail_df = df_classes.loc[df_classes['total_count'] <= 4] # "хвост" из маленьких классов, который просто выучим
print('классов, в которых не больше 4 слов:', tail_df.loc[tail_df['total_count'] <= 4].shape[0])
print('парадигм в этих классах:', tail_df['total_count'].sum())


классов, в которых не больше 4 слов: 248
парадигм в этих классах: 443


In [5]:
# short format to long format
def short_to_long(df):
    print('\n')
    print('short format dataframe with {} lemmas'.format(df.shape[0]))
    #print(df.iloc[0:5,0:5])
    df_long = pd.wide_to_long(df, stubnames = 'form', i = ['lemma','gender'], j = 'formtag')
    df_long = df_long.sample(frac=1, random_state=15) #перемешиваем строки
    print('made long format dataframe with {} wordforms'.format(df_long.shape[0]))
    #print(df_long[:5])
    return df_long

In [13]:
# train test split 
def train_and_test(classes):
    dataset = list(zip([], [], [], [], [], [], [], [], [], [], [], [], [], [], []))
    train = pd.DataFrame(data=dataset, columns=['lemma', 'gender', 'classtag',  
                                                "form1N","form1G","form1D","form1A","form1I","form1L",
                                                "form2N","form2G","form2D","form2A","form2I","form2L"])
    test = train.copy()
    for classtag in classes:
        #print('класс:', classtag)
        paradigms = DF.loc[DF['classtag'] == classtag]
        paradigms = paradigms.sample(frac=1, random_state=12345) #перемешиваем строки
        n = paradigms.shape[0]
        #print('всего парадигм:', n)
        n_train = ceil(n*0.75)
        train_df,test_df = paradigms[:n_train], paradigms[n_train:]
        #print('train: + {} парадигм'.format(train_df.shape[0]))
        train = train.append(train_df)
        #print('test: + {} парадигм\n'.format(test_df.shape[0]))
        test = test.append(test_df)
    train = short_to_long(train)
    test = short_to_long(test)
    return train,test
            

### создаем 2 файла для первого эксперимента: train_full.csv (3/4 от всех классов), test_full.csv для обучения самой полной модели, которую сравним с pymorphy2

In [14]:
%%time
#для каждого класса делим парадигмы на 0.75 трейн и 0.25 тест (классы из хвоста все уходят в трейн)
train, test = train_and_test(cl_list)
test.to_csv('data/test/test_full.csv', sep='\t', encoding = 'utf-8')
train.to_csv('data/train/train_full.csv', sep='\t', encoding = 'utf-8')




short format dataframe with 35192 lemmas
made long format dataframe with 422304 wordforms


short format dataframe with 11528 lemmas
made long format dataframe with 138336 wordforms
Wall time: 16.7 s


In [15]:
train[:3]

,,,classtag,form
lemma,gender,formtag,,
високос,м,2N,м1а,високосы
зимовщик,мо,2N,мо3а,зимовщики
бензель,м,1D,м2а,бензелю


### создаем 1 train файл и 2 test файла для второго эксперимента: train_top, test_top.csv, test_tail.csv для итеративного тестирования моделей при уменьшении обучающей выборки


In [18]:
df_classes[15:18]

,classtag,total_count
15,м2а,478
16,м1а-,452
17,с0,445


In [20]:
#выбрасываем неизменяемые, pluralia tantum и singularia tantum, берем только классы, где не меньше 100 парадигм
df_classes_filtered = df_classes[df_classes.apply(lambda x: not x['classtag'].endswith(('-','+','0')), axis=1)]

In [21]:
df_classes_filtered_top = df_classes_filtered.loc[df_classes['total_count'] >= 100]
cl_list_filtered_top = list(df_classes_filtered_top['classtag']) # 36 классов
df_classes_filtered_tail = df_classes_filtered.loc[df_classes['total_count'] < 100]
cl_list_filtered_tail = list(df_classes_filtered_tail['classtag'])

In [22]:
%%time
#для первых 36 классов делим парадигмы на 0.75 трейн и 0.25 тест, остальные классы модель не видит
train_top, test_top = train_and_test(cl_list_filtered_top)



short format dataframe with 30097 lemmas
made long format dataframe with 361164 wordforms


short format dataframe with 10011 lemmas
made long format dataframe with 120132 wordforms
Wall time: 3.5 s


In [23]:
train_top.to_csv('data/train/train_top.csv', sep='\t', encoding = 'utf-8')
test_top.to_csv('data/test/test_top.csv', sep='\t', encoding = 'utf-8')

In [24]:
#дополнительный тестовый файл с классами, не вошедшими в трейн test_tail.csv
dataset = list(zip([], [], [], [], [], [], [], [], [], [], [], [], [], [], []))
test_tail = pd.DataFrame(data=dataset, columns=['lemma', 'gender', 'classtag', 'form11', 'form12',
                                       'form13', 'form14','form15', 'form16', 'form21', 
                                       'form22', 'form23', 'form24', 'form25','form26'])
for classtag in cl_list_filtered_tail:
    #print('класс:', classtag)
    paradigms = DF.loc[DF['classtag'] == classtag]
    test_tail = test_tail.append(paradigms)
test_tail = short_to_long(test_tail)
test_tail.to_csv('data/test/test_tail.csv', sep='\t', encoding = 'utf-8')



short format dataframe with 2386 lemmas
made long format dataframe with 57264 wordforms


### альтернатива для первого эксперимента. создаем 2 файла : train_full_wordforms.csv (3/4 от всех классов), test_full_wordforms.csv, определяя в тест и трейн каждую словоформу по отдельности. Для обучения самой полной модели, которую сравним с pymorphy2

In [25]:
# train test split 
def train_and_test_wordforms(classes):
    dataset = list(zip([], [], [], [], []))
    train = pd.DataFrame(data=dataset, columns=['lemma', 'gender', 'formtag', 'classtag', 'form'])
    test = train.copy()
    wordforms_long  = short_to_long(DF)
    wordforms_long = wordforms_long.reset_index(level=['lemma','gender','formtag'])
    for classtag in classes:
        #print('класс:', classtag)
        wordforms = wordforms_long.loc[wordforms_long['classtag'] == classtag]
        wordforms = wordforms.sample(frac=1, random_state=12345) #перемешиваем строки
        n = wordforms.shape[0]
        #print('всего словоформ:', n)
        n_train = ceil(n*0.75)
        #n_train = (ceil((n/12)*0.75))*12 если хотим, чтобы классы с <= 4 парадигмами полностью уходили в трейн
        train_df,test_df = wordforms[:n_train], wordforms[n_train:]
        #print('train: + {} словоформ'.format(train_df.shape[0]))
        train = train.append(train_df)
        #print('test: + {} словоформ\n'.format(test_df.shape[0]))
        test = test.append(test_df)
    return train,test
            

In [26]:
%%time
#для каждого класса делим словоформы на 0.75 трейн и 0.25 тест (классы из хвоста все уходят в трейн)
train, test = train_and_test_wordforms(cl_list)
test.to_csv('data/test/test_full_wordforms.csv', sep='\t', encoding = 'utf-8', index = False)
train.to_csv('data/train/train_full_wordforms.csv', sep='\t', encoding = 'utf-8',  index = False)




short format dataframe with 46720 lemmas
made long format dataframe with 560640 wordforms
        lemma gender formtag classtag        form
0   календарь      м      2D      м2в  календарям
1  никонианин     мо      2L   мо1**а  никонианах
Wall time: 58.7 s


In [27]:
test[:5]

,lemma,gender,formtag,classtag,form
159924,каинит,м,2N,м1а,каиниты
530935,гигрограф,м,2L,м1а,гигрографах
31669,фиксатуар,м,1D,м1а,фиксатуару
267882,сельсин,м,1D,м1а,сельсину
474158,радон,м,1I,м1а,радоном
